In [ ]:
import os
import sys
import random
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import Model, load_model
from keras.layers import Input, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img 
from tensorflow import image as tf_image
from tensorflow import data as tf_data
from tensorflow import io as tf_io

In [ ]:
im_width = 512
im_height = 512
im_chan = 1
path_train = '/kaggle/input/small-dataset/Train/'
path_test = '/kaggle/input/small-dataset/Test/'

train_ids = next(os.walk(path_train+"images"))[2]
test_ids = next(os.walk(path_test+"images"))[2]
print(len(train_ids))
print(len(test_ids))

In [ ]:
X_train = np.zeros((len(train_ids), im_height, im_width, 3), dtype=np.uint8) 
Y_train = np.zeros((len(train_ids), im_height, im_width, 1), dtype=bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(train_ids), total=len(train_ids)):
    path = path_train
    img = load_img(path + '/images/' + id_)
    x = img_to_array(img)[:,:,1]
    x = resize(x, (512, 512, 1), mode='constant', preserve_range=True)
    x = np.stack((x[:, :, 0], x[:, :, 0], x[:, :, 0]), axis=2) 
    X_train[n] = x
    mask = img_to_array(load_img(path + '/masks/' + id_))[:,:,1]
    Y_train[n] = resize(mask, (512, 512, 1), mode='constant', preserve_range=True)

print('Done!')

In [ ]:
import keras
from keras import layers
from keras import ops
from glob import glob
from scipy.io import loadmat
from tensorflow.keras import metrics
from tensorflow.keras.utils import register_keras_serializable

@register_keras_serializable()
class MeanIoUMetric(metrics.Metric):
    def __init__(self, num_classes, name='mean_iou', **kwargs):
        super(MeanIoUMetric, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes 
        self.iou_metric = metrics.MeanIoU(num_classes=num_classes) 

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_ = tf.cast(y_pred > 0.5, tf.int32) 
        self.iou_metric.update_state(y_true, y_pred_)

    def result(self):
        return self.iou_metric.result()

    def reset_state(self):
        self.iou_metric.reset_state()
        
    @classmethod
    def from_config(cls, config):
        return cls(**config)  


    def get_config(self):
        config = super().get_config()
        config.update({"num_classes": self.num_classes}) 
        return config
    
mean_iou_metric = MeanIoUMetric(num_classes=2)

from tensorflow.keras.regularizers import l1_l2

def convolution_block(
    block_input,
    num_filters=128,
    kernel_size=3,
    dilation_rate=1,
    use_bias=False,
    dropout_rate=0.2,
    l1_reg=1e-6,
    l2_reg=1e-4,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
        kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)
    )(block_input)
    x = layers.BatchNormalization()(x)
    x = ops.nn.relu(x)
    x = layers.Dropout(dropout_rate)(x)
    return x

def DilatedSpatialPyramidPooling(dspp_input, image_size, dropout_rate=0.2, l1_reg=1e-6, l2_reg=1e-4):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, num_filters=64, kernel_size=1, use_bias=True, dropout_rate=dropout_rate, l1_reg=l1_reg, l2_reg=l2_reg)
    
    out_pool = layers.UpSampling2D(
        size=(dims[-3], dims[-2]), 
        interpolation="bilinear",
    )(x)
    
    out_1 = convolution_block(dspp_input, num_filters=64, kernel_size=1, dilation_rate=1, dropout_rate=dropout_rate, l1_reg=l1_reg, l2_reg=l2_reg)
    out_6 = convolution_block(dspp_input, num_filters=64, kernel_size=3, dilation_rate=6, dropout_rate=dropout_rate, l1_reg=l1_reg, l2_reg=l2_reg)
    out_12 = convolution_block(dspp_input, num_filters=64, kernel_size=3, dilation_rate=12, dropout_rate=dropout_rate, l1_reg=l1_reg, l2_reg=l2_reg)
    
    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12])
    output = convolution_block(x, num_filters=128, kernel_size=1, dropout_rate=dropout_rate, l1_reg=l1_reg, l2_reg=l2_reg)
    return output

def DeeplabV3Plus(image_size, num_classes, dropout_rate=0.2, l1_reg=1e-6, l2_reg=1e-4):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    preprocessed = keras.applications.resnet50.preprocess_input(model_input)
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=preprocessed
    )
    
    x = resnet50.get_layer("conv3_block4_out").output
    x = DilatedSpatialPyramidPooling(x, image_size, dropout_rate, l1_reg, l2_reg)
    
    input_a = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]), 
        interpolation="bilinear",
    )(x)
    
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=32, kernel_size=1, dropout_rate=dropout_rate, l1_reg=l1_reg, l2_reg=l2_reg)
    
    input_b = layers.UpSampling2D(
        size=(input_a.shape[1] // input_b.shape[1], input_a.shape[2] // input_b.shape[2]),
        interpolation="bilinear"
    )(input_b)
    
    x = layers.Concatenate(axis=-1)([input_a, input_b])
    
    x = convolution_block(x, num_filters=64, dropout_rate=dropout_rate, l1_reg=l1_reg, l2_reg=l2_reg)
    
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear"
    )(x)
    
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same", 
                                 kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg))(x)
    return keras.Model(inputs=model_input, outputs=model_output)

IMAGE_SIZE = 512
NUM_CLASSES = 1
DROPOUT_RATE = 0.3
L1_REG = 1e-6
L2_REG = 1e-4

model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES, 
                      dropout_rate=DROPOUT_RATE, l1_reg=L1_REG, l2_reg=L2_REG)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=[mean_iou_metric])
earlystopper = EarlyStopping(patience=10, verbose=1)
checkpointer = ModelCheckpoint('Deeplabv3plus.keras', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.2, batch_size=8, epochs=150,callbacks=[earlystopper,checkpointer])

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('seaborn-ticks')  
plt.figure(figsize=(14, 7))  

plt.plot(results.history['loss'], label='Training Loss', linestyle='-', linewidth=2, color='#3498db')
plt.plot(results.history['mean_iou'], label='Mean IoU', linestyle='-', linewidth=2, color='#2ecc71')
plt.plot(results.history['val_mean_iou'], label='Validation Mean IoU', linestyle='--', linewidth=2, color='#f39c12')


plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss / Mean IoU', fontsize=14)  
plt.title('Training and Validation Metrics', fontsize=16)


plt.legend(fontsize=12, loc='lower left') 


plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)


plt.tight_layout()
plt.ylim(0, 1)

plt.savefig('DeeplabV3+.png', dpi=600)  


plt.show()

In [ ]:
# Get and resize test images and masks
X_test = np.zeros((len(test_ids), im_height, im_width, im_chan), dtype=np.uint8)
Y_test = np.zeros((len(test_ids), im_height, im_width, 1), dtype=bool) 



print('Getting and resizing test images and masks ... ')
sys.stdout.flush()

for n, id_ in tqdm_notebook(enumerate(test_ids), total=len(test_ids)):
    path = path_test
    img = load_img(path + '/images/' + id_)
    x = img_to_array(img)[:,:,1]
    x = np.expand_dims(x, axis=-1)
    X_test[n] = x


    mask = img_to_array(load_img(path + '/masks/' + id_))[:,:,1]
    mask = np.expand_dims(mask, axis=-1)
    Y_test[n] = mask

print('Done!')

In [ ]:
model = tf.keras.models.load_model(
    '/kaggle/working/Deeplabv3plus.keras',
    custom_objects={'MeanIoUMetric': MeanIoUMetric} ,safe_mode=False
)


preds_test = model.predict(X_test, verbose=1)
preds_test_t = (preds_test > 0.2).astype(np.uint8)

In [ ]:
#Perform a sanity check on some random Test Samples
import matplotlib.pyplot as plt
import numpy as np
import random

 
plt.style.use('seaborn-whitegrid') 

fig, axes = plt.subplots(3, 3, figsize=(15, 12))  
fig.subplots_adjust(hspace=0.3, wspace=-0.5)

for i in range(3):
    if i ==0 :
        axes[i, 0].set_title("Image", fontsize=16) 
        axes[i, 1].set_title("Ground Truth", fontsize=16)
        axes[i, 2].set_title("Predicted", fontsize=16)
        
        
    ix = random.randint(0, len(preds_test_t) - 1) 

    # Image
    axes[i, 0].imshow(X_test[ix])
    
    axes[i, 0].axis('off') 

    
    # Ground Truth
    im_gt = axes[i, 1].imshow(Y_test[ix], cmap='inferno')
    
    axes[i, 1].axis('off')
    

    
    # Prediction
    im_pred = axes[i, 2].imshow(preds_test_t[ix], cmap='inferno')
    
    axes[i, 2].axis('off')
    

plt.savefig("prediction_visualization.png", dpi=600) 
plt.show()